In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")
    
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn import metrics
#import eli5
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F
import torch
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import mat4py
import pandas as pd
import data_tracks
import models
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import KFold
from colorama import Fore
import pickle as pkl
from tqdm import tqdm
import train
from style import *
from sklearn import decomposition
import viz

df = data_tracks.get_data() #use_processed=False, use_processed_dicts=False)
n = df.shape[0]
feat_names = data_tracks.get_feature_names(df)
print(feat_names)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['total_displacement', 'mean_square_displacement', 'lifetime', 'center_max', 'left_max', 'right_max', 'up_max', 'down_max', 'X_max', 'X_min', 'X_mean', 'X_std', 'rise', 'fall', 'max_diff', 'min_diff', 'sc_0', 'sc_1', 'sc_2', 'sc_3', 'sc_4', 'sc_5', 'sc_6', 'sc_7', 'sc_8', 'sc_9', 'sc_10', 'sc_11', 'nmf_0', 'nmf_1', 'nmf_2', 'nmf_3', 'nmf_4', 'nmf_5', 'nmf_6', 'nmf_7', 'nmf_8', 'nmf_9', 'nmf_10', 'nmf_11', 'X_max_spl', 'dx_max_spl', 'd2x_max_spl', 'num_local_max_spl', 'num_local_min_spl']


# loop over classifiers

In [9]:
outcome_def = 'y_consec_sig'
out_dir = f'results/outcome={outcome_def}'
os.makedirs(out_dir, exist_ok=True)
for feature_selection in ['select_rf']: #, 'select_lasso']: # select_lasso
    for balancing in ['ros']: # none
        for model_type in tqdm(['irf', 'logistic', 'dt', 'rf', 'mlp2', 'dt', 'svm']): #,'gb', 'logistic', 'dt', 'svm', 'gb', rf', 'mlp2', 'irf']):
            for num_feats in [15, 25, len(feat_names)]: # number of total features to consider
                for feature_selection_num in [3, 5, 7, 11, 15]: # number of feature to select [4, 9, 11, 23, 35, 39]
                    feats = feat_names[:num_feats]
                    out_name = f'{model_type}_{num_feats}_{balancing}_{feature_selection}={feature_selection_num}'
                    #print(out_name)
                    train.train(df, feat_names=feats, model_type=model_type, 
                                balancing=balancing, outcome_def=outcome_def,
                                feature_selection=feature_selection,
                                feature_selection_num=feature_selection_num,
                                out_name=f'{out_dir}/{out_name}.pkl')

100%|██████████| 7/7 [03:00<00:00, 29.39s/it]


# dim reduction with classifier

In [3]:
# feats_to_reduce = feat_names[:11] 
feats_to_reduce = feat_names
X = df[feat_names]
X = (X - X.mean()) / X.std()
y = df['y_thresh'].values

**look at pcs**

In [ ]:
# perform pca
pca = decomposition.PCA(whiten=True)
# pca = decomposition.SparsePCA()
# pca = decomposition.NMF()
pca.fit(X)
plt.figure(figsize=(6, 9), dpi=200)
# plt.figure(figsize=(6, 5), dpi=200)
viz.plot_pcs(pca, X)

In [ ]:
pca = decomposition.PCA(n_components=2)
X_reduced = pca.fit_transform(X)
plt.figure(dpi=200)
ys_neg = df['y_thresh'].values == 0
plt.plot(X_reduced[:, 0][ys_neg], X_reduced[:, 1][ys_neg], 'o', 
         color=cr, alpha=0.3, markeredgewidth=0, ms=2) #, c=df['y_thresh'].values, alpha=0.1)
plt.plot(X_reduced[:, 0][~ys_neg], X_reduced[:, 1][~ys_neg], 'o', 
         color=cb, alpha=0.3, markeredgewidth=0, ms=2)
# plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=df['y_thresh'].values, alpha=0.1, cmap={0:'red', 1:'blue'})
plt.xlabel('PC1')
plt.ylabel('PC2')
#             cdf[np.arange(X.shape[0]), cmap='viridis', alpha=0.1)
plt.show()